In [1]:
import pandas as pd
trips_by_nta = pd.read_csv('tripsonly.csv')
trips_by_nta[:3]

,Taxi Zone,NTA,Ride Count
0,2,QN99,174
1,3,BX31,163797
2,4,MN28,482278


In [2]:
# Data cleansing --> checking dataset for outlier zones
# It turns out XX99 zones are aggregates of park and cemetery zones, so we should remove them
zones_del = []
for i,r in trips_by_nta.iterrows():
    if ('99' in r['NTA']):
        zones_del.append(i)
        print(r.NTA)

QN99
QN99
MN99
BX99
BK99
MN99
BX99
QN99
QN99
MN99
MN99
MN99
SI99
BK99
MN99
MN99
BK99
BX99
BK99
MN99
QN99
BX99
QN99


In [3]:
trips_by_nta['Ride Count'][zones_del].sum()

971655

In [4]:
# This is Central Park, was assigned to MN99 based on Shapely
trips_by_nta.loc[41]

Taxi Zone         43
NTA             MN99
Ride Count    402928
Name: 41, dtype: object

In [5]:
dfn = trips_by_nta.drop(index = zones_del)
len(trips_by_nta)

262

In [6]:
len(dfn)

239

In [7]:
dfn.to_csv('tripsx99.csv', index=False)

In [8]:
dfn.drop_duplicates(subset='NTA', keep=False, inplace=True)
len(dfn)

160

In [9]:
dfn = dfn.set_index(['NTA'])
dfn[:3]

,Taxi Zone,Ride Count
NTA,,
BX31,3,163797
SI48,5,17136
SI14,6,31369


In [10]:
acs = pd.read_excel("https://www1.nyc.gov/assets/planning/download/office/data-maps/nyc-population/acs/econ_2016acs5yr_nta.xlsx?r=1")
acs[:3]

,GeoType,GeogName,GeoID,Borough,Pop16plE,Pop16plM,Pop16plC,Pop16plP,Pop16plZ,LFE,...,Pv400t499E,Pv400t499M,Pv400t499C,Pv400t499P,Pv400t499Z,Pv500plE,Pv500plM,Pv500plC,Pv500plP,Pv500plZ
0,NTA2010,Allerton-Pelham Gardens,BX31,Bronx,26656,1076.0,2.5,100.0,NaN,15885,...,3387,402.0,7.2,11.1,1.2,8233,669.0,4.9,26.9,1.9
1,NTA2010,Bedford Park-Fordham North,BX05,Bronx,42293,1424.0,2.0,100.0,NaN,27301,...,3957,584.0,9.0,7.1,1.0,3763,528.0,8.5,6.8,0.9
2,NTA2010,Belmont,BX06,Bronx,22085,1070.0,2.9,100.0,NaN,12311,...,805,266.0,20.1,3.0,1.0,1212,274.0,13.7,4.6,1.0


In [11]:
drop = []
for i,r in acs.iterrows():
    if '99' in r['GeoID']:
        drop.append(i)
drop

[21, 76, 108, 158, 188]

In [12]:
acs = acs.drop(index = drop)
len(acs)

190

In [13]:
acs = acs.set_index('GeoID')
acs[:3]

,GeoType,GeogName,Borough,Pop16plE,Pop16plM,Pop16plC,Pop16plP,Pop16plZ,LFE,LFM,...,Pv400t499E,Pv400t499M,Pv400t499C,Pv400t499P,Pv400t499Z,Pv500plE,Pv500plM,Pv500plC,Pv500plP,Pv500plZ
GeoID,,,,,,,,,,,,,,,,,,,,,
BX31,NTA2010,Allerton-Pelham Gardens,Bronx,26656,1076.0,2.5,100.0,NaN,15885,880.0,...,3387,402.0,7.2,11.1,1.2,8233,669.0,4.9,26.9,1.9
BX05,NTA2010,Bedford Park-Fordham North,Bronx,42293,1424.0,2.0,100.0,NaN,27301,1221.0,...,3957,584.0,9.0,7.1,1.0,3763,528.0,8.5,6.8,0.9
BX06,NTA2010,Belmont,Bronx,22085,1070.0,2.9,100.0,NaN,12311,942.0,...,805,266.0,20.1,3.0,1.0,1212,274.0,13.7,4.6,1.0


In [15]:
#Commute to work: % drive alone
drv_aln = acs['CW_DrvAlnP']
#Commute to work: % carpool
carpool = acs['CW_CrpldP']
#Commute to work: public transit
pub_trns = acs['CW_PbTrnsP']
#Median income
incomes = acs['MdFamIncE']

In [17]:
dfn['Median Income'] = incomes
dfn[:5]

,Taxi Zone,Ride Count,Median Income
NTA,,,
BX31,3,163797,69099.0
SI48,5,17136,96734.0
SI14,6,31369,83357.0
QN70,7,1002109,61469.0
QN48,9,76735,85282.0


In [18]:
dfn.dropna(inplace = True)
len(dfn)

159

In [19]:
dfn.to_csv('fhvincome.csv')

In [50]:
url = 'http://web.mta.info/nyct/facts/ridership/ridership_bus.htm'
dfb = pd.read_html(url)

In [51]:
dfb[2][-10:]

,Route,Note,2012,2013,2014,2015,2016,2017,2016-2017 Change,2016-2017 Change.1,2017 Rank
239,Exp Special/Adj,NaN,49,-203,99,90,107,73,-34,-31.8%,NaN
240,Express Total,NaN,42798,42658,42195,40543,40817,40109,-708,-1.7%,NaN
241,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
242,Brooklyn Total,NaN,648169,640810,627168,615477,612752,581227,-31525,-5.1%,NaN
243,Bronx Total,NaN,529446,542716,548216,538711,522826,483663,-39163,-7.5%,NaN
244,Manhattan Total,NaN,488821,476800,447585,424707,410479,380075,-30404,-7.4%,NaN
245,Queens Total,NaN,373621,373258,366158,358402,359768,350548,-9220,-2.6%,NaN
246,Staten Is Total,NaN,126044,128980,129631,128997,128042,123960,-4082,-3.2%,NaN
247,Adjustments,NaN,3208,3812,4334,4092,4252,4520,+268,+6.3%,NaN
248,Grand Total,NaN,2169311,2166376,2123092,2070386,2038119,1923993,-114126,-5.6%,NaN


In [59]:
bus_df = dfb[2][-7:].loc[:,'Route':'2017']
bus_df

,Route,Note,2012,2013,2014,2015,2016,2017
242,Brooklyn Total,NaN,648169,640810,627168,615477,612752,581227
243,Bronx Total,NaN,529446,542716,548216,538711,522826,483663
244,Manhattan Total,NaN,488821,476800,447585,424707,410479,380075
245,Queens Total,NaN,373621,373258,366158,358402,359768,350548
246,Staten Is Total,NaN,126044,128980,129631,128997,128042,123960
247,Adjustments,NaN,3208,3812,4334,4092,4252,4520
248,Grand Total,NaN,2169311,2166376,2123092,2070386,2038119,1923993


In [60]:
bus_df.reset_index(inplace = True)
bus_df

,index,Route,Note,2012,2013,2014,2015,2016,2017
0,242,Brooklyn Total,NaN,648169,640810,627168,615477,612752,581227
1,243,Bronx Total,NaN,529446,542716,548216,538711,522826,483663
2,244,Manhattan Total,NaN,488821,476800,447585,424707,410479,380075
3,245,Queens Total,NaN,373621,373258,366158,358402,359768,350548
4,246,Staten Is Total,NaN,126044,128980,129631,128997,128042,123960
5,247,Adjustments,NaN,3208,3812,4334,4092,4252,4520
6,248,Grand Total,NaN,2169311,2166376,2123092,2070386,2038119,1923993


In [61]:
bus_df.drop(columns = ['index','Note'], inplace = True)
bus_df.drop(index = 5, inplace = True)
bus_df

,Route,2012,2013,2014,2015,2016,2017
0,Brooklyn Total,648169,640810,627168,615477,612752,581227
1,Bronx Total,529446,542716,548216,538711,522826,483663
2,Manhattan Total,488821,476800,447585,424707,410479,380075
3,Queens Total,373621,373258,366158,358402,359768,350548
4,Staten Is Total,126044,128980,129631,128997,128042,123960
6,Grand Total,2169311,2166376,2123092,2070386,2038119,1923993


In [62]:
bus_df.to_csv('yearly_busbyboro.csv', index = False)